In [ ]:
# -*- coding: utf-8 -*-
# Python 3.5
# combodia

In [2]:
# load file to list 'sentences' split by sentence

import re
caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def to_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n", " ")
    text = re.sub(prefixes, "\\1<prd>", text)
    text = re.sub(websites, "<prd>\\1", text)
    if "Ph.D" in text: text = text.replace("Ph.D.", "Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] ", " \\1<prd> ", text)
    text = re.sub(acronyms+" "+starters, "\\1<stop> \\2", text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]", "\\1<prd>\\2<prd>\\3<prd>", text)
    text = re.sub(caps + "[.]" + caps + "[.]", "\\1<prd>\\2<prd>", text)
    text = re.sub(" "+suffixes+"[.] "+starters, " \\1<stop> \\2", text)
    text = re.sub(" "+suffixes+"[.]", " \\1<prd>", text)
    text = re.sub(" " + caps + "[.]", " \\1<prd>", text)
    if "”" in text: text = text.replace(".”", "”.")
    if "\"" in text: text = text.replace(".\"", "\".")
    if "!" in text: text = text.replace("!\"", "\"!")
    if "?" in text: text = text.replace("?\"", "\"?")
    text = text.replace(".", ".<stop>")
    text = text.replace("?", "?<stop>")
    text = text.replace("!", "!<stop>")
    text = text.replace("<prd>", ".")
    text = text.replace(",  ", ", ")
    text = text.replace("  ", " ")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [28]:
# "NSDP.txt" is transformed by Cambodia's "National Strategic Development Plan 2014-2018 EN Final.pdf"
# using Adobe

dirc = r'C:\\Users\\DD\\OneDrive\\Documents\\Rapid_Assessment_Tools\\Plans_test\\cambodia\\'

In [2]:
import pandas as pd
# golden standard
cambodia_gs = pd.ExcelFile(r'C:\\Users\\DD\\OneDrive\\Documents\\Rapid_Assessment_Tools\\Features\\Cambodia Reviewed.xlsx')
# gs.sheet_names
gs = cambodia_gs.parse("Sheet1")
gs.head()

,No.,Doc,Goal,Page,Content
0,1.0,NSDP,1.1,105.0,Achieving more than 1 percentage point reducti...
1,2.0,NSDP,1.2,105.0,Achieving more than 1 percentage point reducti...
2,3.0,NSDP,1.3,193.0,strengthening the social protection system to ...
3,4.0,NSPS,1.3,NaN,Expand coverage of IDPoor to all rural househo...
4,5.0,NaN,1.4,NaN,"Intensify land reforms, focusing on strengthen..."


In [3]:
# extract goldent standard by input goal number
def golden_standard(df, goal:int):
    g = df.loc[df['Goal'] == goal, 'Content']
    i = 0
    l = []
    while i < len(g):
        l.append(g.iloc[i])
        i += 1
    return l

In [109]:
goal_no = 1.1

In [110]:
gs_list = golden_standard(gs, goal_no)
gs_list

['Achieving more than 1 percentage point reduction in poverty incidence annually']

In [111]:
# find golden standars in which item in matched list, if exist
def find_gs(gs_list, matched_list):
    l = []
    for sentence in matched_list:
        for gs in gs_list:
            if gs.lower() in sentence:
                l.append(matched_list.index(sentence))
    if len(l) == 0:
        return None
    return l

In [112]:
# goals
sdg_goal = pd.ExcelFile(r'C:\\Users\\DD\\OneDrive\\Documents\\Rapid_Assessment_Tools\\Features\\SDG_Goal.xlsx')
# sheet_names
goals = sdg_goal.parse("Sheet1")
goals.head()

,Number,Content
0,1.1,"By 2030, eradicate extreme poverty for all peo..."
1,1.2,"By 2030, reduce at least by half the proportio..."
2,1.3,Implement nationally appropriate social protec...
3,1.4,"By 2030, ensure that all men and women, in par..."
4,1.5,"By 2030, build the resilience of the poor and ..."


In [113]:
# target goal, for example 1.3
goal = goals.loc[goals['Number'] == goal_no, 'Content'].item()
goal

'By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day'

In [114]:
# calculate similarity, TFiDF
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df=1)

def goal_similarity(matched_list, goal, ind_list = None):
    l = matched_list[:]
    l.insert(0, goal.lower())
    tfidf = vect.fit_transform(l)
    t = (tfidf * tfidf.T).A
    ls = []
    i = 0
    if ind_list:
        j = 0
        while j < len(ind_list):
            threshold = t[0][ind_list[j]+1]
            while i < len(t):
                if t[0][i] >= threshold:
                    ls.append([i, t[j][i], l[i]])
                i += 1
            j += 1
        # ls = list(set(ls))
        return ls
    if not ind_list:
        while i < len(t):
            ls.append([i, t[0][i], l[i]])
            i += 1
        return ls

In [115]:
# matched result 1
result_judy = pd.ExcelFile(r'C:\\Users\\DD\\OneDrive\\Documents\\Rapid_Assessment_Tools\\Features\\output_judy.xlsx')
# sheet_names
result = result_judy.parse("Sheet1")
result.head()

,sector,sent_id,sector_sent,p,tokens,1.1,1.2,1.3,1.4,1.5
0,1.0,0,1_0,Education Sector,"[u'educ', u'sector', u'Education Sector']",0.0,0.0,0.0,0.0,0.0
1,NaN,1,1_1,The single most important factor towards achie...,"[u'the', u'singl', u'most', u'import', u'facto...",0.0,0.0,0.0,0.0,0.0
2,NaN,2,1_2,Education virtually impacts all dimensions of ...,"[u'educ', u'virtual', u'impact', u'all', u'dim...",1.0,1.2,0.0,0.0,1.3
3,NaN,3,1_3,An educated citizenry that is able to understa...,"[u'an', u'educ', u'citizenri', u'that', u'is',...",0.0,0.0,0.0,0.0,0.0
4,NaN,4,1_4,"Similarly, the realization of our vision of a ...","[u'similarli', u',', u'the', u'realiz', u'of',...",0.0,0.0,0.0,0.0,0.0


In [119]:
p_list = list(result.loc[result[str(goal_no)] >= 1, 'p'])
len(p_list)

121

In [120]:
# check if golden standard is in matched list
ind_list = find_gs(gs_list, p_list)
ind_list

In [121]:
# sometimes, 'True' because did not stem
ind_list == None

True

In [122]:
# if golden standard is not in matched list, function "goal_similarity" will not filter but sorting 
ls = goal_similarity(p_list, goal, ind_list)
len(ls)

122

In [91]:
ls[:10]

[[0,
  0.99999999999999989,
  'by 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day'],
 [1,
  0.13492451767552829,
  'Education virtually impacts all dimensions of development and is critical for ensuring sustainable poverty alleviation which can only come through the empowerment of our people, particularly the remote and most vulnerable sections of our population'],
 [2,
  0.024376020097355726,
  'Provision of monastic education is facilitated by the central monastic body and privately managed independent monasteries'],
 [3,
  0.016223293782196332,
  'Tertiary education in Bhutan comprises of three years of undergraduate studies provided by the Royal University of Bhutan (RUB) through its 10 constituent colleges located in various parts of the country and one private college, the Royal Thimphu College'],
 [4,
  0.0091780073997876615,
  'This 1.5 percent are assumed to be children in remote and hard to reach areas, c

In [123]:
df = pd.DataFrame(ls, columns=['index', 'similarity', 'sentence'])

In [33]:
s = df['sentence'][0]

In [56]:
result

,sector,sent_id,p,1.1,1.2,1.3,1.4,1.5,2.1,2.2,...,16.1,16.2,16.3,16.4,16.5,16.6,16.7,16.8,16.9,16.10
0,1.0,0,Education Sector,0.0,0,0.0,0.0,0.0,0,0,...,0,0,0,0,0.0,0,0.0,0,0,0
1,NaN,1,The single most important factor towards achie...,0.0,0,0.0,1.1,0.0,0,0,...,0,0,0,0,0.0,0,0.0,0,0,0
2,NaN,2,Education virtually impacts all dimensions of ...,0.0,0,0.0,0.0,1.1,0,0,...,0,0,0,0,0.0,0,0.0,0,0,0
3,NaN,3,An educated citizenry that is able to understa...,0.0,0,0.0,0.0,0.0,0,0,...,0,0,0,0,0.0,0,0.0,0,0,0
4,NaN,4,"Similarly, the realization of our vision of a ...",0.0,0,0.0,0.0,0.0,0,0,...,0,0,0,0,0.0,0,0.0,0,0,0
5,NaN,5,"Recognizing this, the education sector continu...",0.0,0,0.0,0.0,0.0,0,0,...,0,0,0,0,0.0,0,0.0,0,0,0
6,NaN,6,Education in Bhutan comprises early childcare ...,0.0,0,0.0,1.0,0.0,0,0,...,0,0,0,0,0.0,0,0.0,0,0,0
7,NaN,7,The responsibility for the administration of e...,0.0,0,0.0,0.0,0.0,0,0,...,0,0,0,0,0.0,0,0.0,0,0,0
8,NaN,8,Provision of monastic education is facilitated...,0.0,0,1.0,0.0,0.0,0,0,...,0,0,0,0,0.0,0,0.0,0,0,0
9,NaN,9,Although education plans are centrally coordin...,0.0,0,0.0,0.0,0.0,0,0,...,0,0,0,0,0.0,0,0.0,0,0,0


In [106]:
i = 1
result_1 = result.copy()
while i < len(df['sentence']):
    s = df['sentence'][i]
    sml = float(df['similarity'][i])
    # result_1['1.1'] = ''
    result_1.loc[result_1['p'] == s, '1.2'] = sml
    i += 1

In [98]:
s = df['sentence']
s

0      by 2030, eradicate extreme poverty for all peo...
1      Education virtually impacts all dimensions of ...
2      Provision of monastic education is facilitated...
3      Tertiary education in Bhutan comprises of thre...
4      This 1.5 percent are assumed to be children in...
5      On the other hand Bhutan continues to face an ...
6      This has resulted in a paradoxical situation o...
7      The fact that most of our students are unable ...
8      Some of the key deficiencies in the quality of...
9      The overall level of performance of students i...
10     Graduates lack basic analytical and communicat...
11     Schools lack quality processes for developing ...
12     The report underscored three critical reasons ...
13     However, our education system continues to be ...
14     Some of the reasons for this include lack of i...
15     The possible introduction of a vouchers scheme...
16     Due to lack of economies of scale the difficul...
17     Similarly, the downgrade

In [77]:
sm = df['similarity'][0]
type(sm)

numpy.float64

In [107]:
result_1.head()

,sector,sent_id,sector_sent,p,tokens,1.1,1.2,1.3,1.4,1.5
0,1.0,0,1_0,Education Sector,"[u'educ', u'sector', u'Education Sector']",0.0,0.000000,0.0,0.0,0.0
1,NaN,1,1_1,The single most important factor towards achie...,"[u'the', u'singl', u'most', u'import', u'facto...",0.0,0.000000,0.0,0.0,0.0
2,NaN,2,1_2,Education virtually impacts all dimensions of ...,"[u'educ', u'virtual', u'impact', u'all', u'dim...",1.0,0.149298,0.0,0.0,1.3
3,NaN,3,1_3,An educated citizenry that is able to understa...,"[u'an', u'educ', u'citizenri', u'that', u'is',...",0.0,0.000000,0.0,0.0,0.0
4,NaN,4,1_4,"Similarly, the realization of our vision of a ...","[u'similarli', u',', u'the', u'realiz', u'of',...",0.0,0.000000,0.0,0.0,0.0


In [125]:
result_1.to_excel(dirc+'result_test2.xlsx')

In [100]:
result_1['1.1']

0       0.000000
1       0.000000
2       0.134925
3       0.000000
4       0.000000
5       0.000000
6       0.000000
7       0.000000
8       0.024376
9       0.000000
10      0.000000
11      0.000000
12      0.000000
13      0.000000
14      0.000000
15      0.000000
16      0.000000
17      0.000000
18      0.000000
19      0.016223
20      0.000000
21      0.000000
22      0.000000
23      0.000000
24      0.000000
25      0.000000
26      0.000000
27      0.000000
28      0.000000
29      0.000000
          ...   
1219    0.000000
1220    0.000000
1221    0.000000
1222    0.000000
1223    0.000000
1224    0.000000
1225    0.000000
1226    0.000000
1227    0.000000
1228    0.000000
1229    0.000000
1230    0.000000
1231    0.000000
1232    0.000000
1233    0.000000
1234    0.000000
1235    0.000000
1236    0.000000
1237    0.000000
1238    0.000000
1239    0.000000
1240    0.000000
1241    0.000000
1242    0.000000
1243    0.000000
1244    0.000000
1245    0.000000
1246    0.0000